## Import packages

In [2]:
%matplotlib widget

In [3]:
import os
import glob
import numpy  as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import ipywidgets as widgets
import wave

### Choose file to inspect

In [4]:
directory='/Users/saroltagabulya/git/Orca/'
wavs=glob.glob('wav_files/*.wav')
file=[]

dropdown=widgets.Dropdown(
    options=wavs,
    description='Wav files:',
    disabled=False,
)

def dropdown_eventhandler(change):
    file.clear()
    file.append(change.new)
    print(change.new)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

Dropdown(description='Wav files:', options=('wav_files/2015-11-10--15-25.wav', 'wav_files/2017-02-04--10-14-06…

wav_files/2017-02-04--10-14-06--00-16-35--BC.wav


### Read metadata


In [6]:
# Set sample rate
try:
    wave_file=wave.open(file[0], "rb")
    sample_rate = wave_file.getframerate()
except:
    sample_rate=int(input('Please check sampling rate manually in the metadata file and set below in Hz! \n'))
    
# Set reference value
ref_value=int(input('Please check reference value in the files README and set below in uPa! \n'))

# Calibration value
cal_value=int(input('Please check calibration value in the files README and set below! \n what does the fullscale voltage correspond to in Pa? \n '))


# Set researcher
researcher= input('Please indicate the researcher who performs the crop by initials [SG, JR] \n ')

Please check sampling rate manually in the metadata file and set below in Hz! 
100000
Please check reference value in the files README and set below in uPa! 
1
Please check calibration value in the files README and set below! 
 what does the fullscale voltage correspond to in Pa? 
 1
Please indicate the researcher who performs the crop by initials [SG, JR] 
 SG


### Read in wav file

In [7]:
directory='/Users/saroltagabulya/git/Orca/call_segments/'

folder_name = directory + file[0][10:-4]
os.chdir(folder_name)
wavs=glob.glob('*.wav')
wavs

['2017-02-04--10-14-06--00-16-35--BC_97052456_97622778.wav',
 '2017-02-04--10-14-06--00-16-35--BC_86994040_87290169.wav',
 '2017-02-04--10-14-06--00-16-35--BC_92602541_93030283.wav',
 '2017-02-04--10-14-06--00-16-35--BC_39943424_40415037.wav',
 '2017-02-04--10-14-06--00-16-35--BC_85157436_85135501.wav',
 '2017-02-04--10-14-06--00-16-35--BC_40985359_41544714.wav',
 '2017-02-04--10-14-06--00-16-35--BC_87915331_88178557.wav',
 '2017-02-04--10-14-06--00-16-35--BC_94554799_94730283.wav',
 '2017-02-04--10-14-06--00-16-35--BC_95196915_95668528.wav',
 '2017-02-04--10-14-06--00-16-35--BC_51657968_51921194.wav',
 '2017-02-04--10-14-06--00-16-35--BC_85629049_85958082.wav',
 '2017-02-04--10-14-06--00-16-35--BC_84192275_84576146.wav']

### Calculate spectogram

In [8]:
def open_wav(wav, sample_rate):
    y, sr = librosa.load(wav, sr=sample_rate)
    return y

def convert_wav_to_spect(y, ref_value):
    
    if len(y)>0:
        # Convert to spectogram 
        Y = librosa.stft(y, ) 
        Ydb = librosa.amplitude_to_db(abs(Y), ref=ref_value) # Try if it works leaving away abs!
                                                   # Check whether reference should go in as amplitude or power
        return Ydb
    else: 
        print ('Error with wav')
        return None

### Event handling function

In [9]:
def onclick_for_w(w):
    def onclick(event):
        if event.key=='b':
            row.clear()
            events.append(event)
            row.append(event.xdata + (s)*10*60)
            plt.gca().set_title('onset: {}'.format(event.xdata))

        elif event.key=='e':
            events.append(event)
            row.append(event.xdata + (s)*10*60)
            plt.gca().set_title('offset: {}'.format(event.xdata))

            if len(row)==2:
                row.append(filename)
                time_stamps.loc[len(time_stamps)] = row
                plt.gca().set_title('Timestamps appended. \n onset: {} \n offset {}'.format(onset, offset))
                row.clear()

        elif event.key == 's':
            # save croppings
            print('saved dataframe')
            time_stamps.to_csv('time_stamps.csv')

        else:
            plt.gca().set_title('Could not interpret user input, please make a new selection:\n b: define onset timestamps \n e: define offset timestamp and add to df \n s: save df to csv file'.format(onset, offset))
    return onclick

### Initializing data structures

In [32]:
#amplitudes=pd.DataFrame()
events = []
row=[]

### Close previous plots and replot interactive plots

In [33]:
plt.close()

In [40]:
for w in [1, 2]: #list(range(len(wavs))):
    wav=wavs[w]
    y=open_wav(wav, sample_rate)
    Ydb=convert_wav_to_spect(y, ref_value)
    
    if type(Ydb)!= None: 
        
        try: 
            plt.figure()

            # tell mpl_connect we want to pass a 'button_press_event' into onclick when the event is detected
            plt.gcf().canvas.mpl_connect('key_press_event', onclick_for_w(w))

            librosa.display.specshow(Ydb, cmap='seismic', sr=sample_rate, x_axis='time', y_axis='hz')
            plt.xlabel('time(samples)')
            plt.ylabel('dBa relative to 1 uPa**2/Hz')
            plt.colorbar()
            plt.show()
            print('succeeded to open {}'.format(w))
            
        except:
                print('could not display spectogram')
    else: 
        print('problem with wav')
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

succeeded to open 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

succeeded to open 2


In [ ]:
plt.show()

In [1]:
hop_length=256, n_fft=4096

NameError: name 'plt' is not defined

In [13]:
sample_rate

100000

0.014251709

In [35]:
y=open_wav('2017-02-04--10-14-06--00-16-35--BC_39943424_40415037.wav', sample_rate)

In [36]:
y

array([0.00018311, 0.00030518, 0.00033569, ..., 0.00152588, 0.00167847,
       0.0017395 ], dtype=float32)

['2017-02-04--10-14-06--00-16-35--BC_97052456_97622778.wav',
 '2017-02-04--10-14-06--00-16-35--BC_86994040_87290169.wav',
 '2017-02-04--10-14-06--00-16-35--BC_92602541_93030283.wav',
 '2017-02-04--10-14-06--00-16-35--BC_39943424_40415037.wav',
 '2017-02-04--10-14-06--00-16-35--BC_85157436_85135501.wav',
 '2017-02-04--10-14-06--00-16-35--BC_40985359_41544714.wav',
 '2017-02-04--10-14-06--00-16-35--BC_87915331_88178557.wav',
 '2017-02-04--10-14-06--00-16-35--BC_94554799_94730283.wav',
 '2017-02-04--10-14-06--00-16-35--BC_95196915_95668528.wav',
 '2017-02-04--10-14-06--00-16-35--BC_51657968_51921194.wav',
 '2017-02-04--10-14-06--00-16-35--BC_85629049_85958082.wav',
 '2017-02-04--10-14-06--00-16-35--BC_84192275_84576146.wav']

### Try out different spectogram parameters

In [165]:
def open_wav(wav, sample_rate):
    y, sr = librosa.load(wav, sr=sample_rate)
    return y

def convert_wav_to_spect(y, ref_value):
    
    if len(y)>0:
        # Convert to spectogram 
        Y = librosa.stft(y, n_fft=1024, hop_length=32, win_length=500) 
        Ydb = librosa.amplitude_to_db(abs(Y), ref=ref_value) # Try if it works leaving away abs!
                                                   # Check whether reference should go in as amplitude or power
        return Ydb
    else: 
        print ('Error with wav')
        return None

In [166]:
wav='2017-02-04--10-14-06--00-16-35--BC_97052456_97622778.wav'
y=open_wav(wav, sample_rate)
Ydb=convert_wav_to_spect(y, ref_value)
    
#if type(Ydb)!= None: 
        

#plt.figure(figsize=(round(np.shape(Ydb)[0]/10), round(np.shape(Ydb)[1]/10)))
plt.figure(figsize=(10, 5))

# tell mpl_connect we want to pass a 'button_press_event' into onclick when the event is detected
#plt.gcf().canvas.mpl_connect('key_press_event', onclick_for_w(w))

librosa.display.specshow(Ydb, cmap='seismic', sr=sample_rate, x_axis='time', y_axis='fft')
plt.xlabel('time(samples)')
plt.ylabel('dBa relative to 1 uPa**2/Hz')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [167]:
plt.savefig('#21_nfft_1024_hop_32_win_500.png')